In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [61]:
# set a rsandom seed for reproducibility
torch.manual_seed(42)

In [62]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [63]:
df = pd.read_csv('fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [64]:
# train test split

x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [65]:
X_train , X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [66]:
#scaling the features

X_train = X_train/255.0
X_test = X_test/255.0

In [67]:
# Create a CustomDataset class
class CustomDataset(Dataset):
    def __init__(self, Features, Labels):
        self.Features = torch.tensor(Features, dtype=torch.float32)
        self.Labels = torch.tensor(Labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.Features)
    
    def __getitem__(self, idx):
        return self.Features[idx], self.Labels[idx]


In [68]:
# create a train_dataset object 
train_dataset = CustomDataset(X_train, y_train)

In [69]:
len(train_dataset)

4800

In [70]:
# create test_dataset object 
test_dataset = CustomDataset(X_test, y_test)

In [71]:
len(test_dataset)

1200

In [72]:
# create and test loader objects
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [73]:
# Define the NN class

class MyNN(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        return self.model(x)

In [74]:
# set learning rate and epochs
lr = 0.1
epochs = 100

In [75]:
# Instantiate the model
model = MyNN(X_train.shape[1])
model.to(device)

# loss function 
criterion = nn.CrossEntropyLoss()

# using the best optimizer for the model - adam / sgd / rmsprop
optimizer = optim.SGD(model.parameters(), lr=lr)

In [76]:
# training loop 

for epoch in range(epochs):
    output = model.train()
    for features, labels in train_loader:
        features = features.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/100, Loss: 0.7689862847328186
Epoch 2/100, Loss: 0.5638744235038757
Epoch 3/100, Loss: 0.6357843279838562
Epoch 4/100, Loss: 0.44449886679649353
Epoch 5/100, Loss: 0.45776283740997314
Epoch 6/100, Loss: 0.4718804657459259
Epoch 7/100, Loss: 0.4849931597709656
Epoch 8/100, Loss: 0.2724159359931946
Epoch 9/100, Loss: 0.29381412267684937
Epoch 10/100, Loss: 0.16522419452667236
Epoch 11/100, Loss: 0.5025491714477539
Epoch 12/100, Loss: 0.482719749212265
Epoch 13/100, Loss: 0.616004467010498
Epoch 14/100, Loss: 0.7262017726898193
Epoch 15/100, Loss: 0.3375810980796814
Epoch 16/100, Loss: 0.26656830310821533
Epoch 17/100, Loss: 0.17142505943775177
Epoch 18/100, Loss: 0.3644021451473236
Epoch 19/100, Loss: 0.4625479578971863
Epoch 20/100, Loss: 0.28775015473365784
Epoch 21/100, Loss: 0.2589673399925232
Epoch 22/100, Loss: 0.2872511148452759
Epoch 23/100, Loss: 0.19640500843524933
Epoch 24/100, Loss: 0.12837988138198853
Epoch 25/100, Loss: 0.1472007781267166
Epoch 26/100, Loss: 0.17158

In [77]:
# set the model to evaluation mode
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for features, labels in test_loader:
        features = features.to(device)
        labels = labels.to(device)
        outputs = model(features)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy: {correct/total}')

Accuracy: 0.835


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer

# Load dataset
df = pd.read_csv('fmnist_small.csv')

# Train-test split
x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Scale the features
X_train = X_train / 255.0
X_test = X_test / 255.0

# Create a CustomDataset class
class CustomDataset(Dataset):
    def __init__(self, Features, Labels):
        self.Features = torch.tensor(Features, dtype=torch.float32)
        self.Labels = torch.tensor(Labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.Features)
    
    def __getitem__(self, idx):
        return self.Features[idx], self.Labels[idx]

# Create train and test datasets
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

# Define the Neural Network Model
class MyNN(pl.LightningModule):
    def __init__(self, input_size, learning_rate=0.1):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10))
        self.learning_rate = learning_rate
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        features, labels = batch
        outputs = self(features)
        loss = self.criterion(outputs, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        features, labels = batch
        outputs = self(features)
        loss = self.criterion(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == labels).float().mean()
        self.log('val_loss', loss)
        self.log('val_accuracy', accuracy)
        return loss

    def test_step(self, batch, batch_idx):
        features, labels = batch
        outputs = self(features)
        loss = self.criterion(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == labels).float().mean()
        self.log('test_loss', loss)
        self.log('test_accuracy', accuracy)
        return loss

    def configure_optimizers(self):
        return optim.SGD(self.parameters(), lr=self.learning_rate)

# Initialize the model
model = MyNN(input_size=X_train.shape[1])

# Initialize the PyTorch Lightning Trainer
trainer = Trainer(max_epochs=100, accelerator='gpu' if torch.cuda.is_available() else 'cpu')

# Train the model
trainer.fit(model, train_loader, test_loader)

# Test the model
trainer.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | Sequential       | 109 K  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
109 K     Trainable params
0         Non-trainable params
109 K     Total params
0.438     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


c:\Users\abhin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\abhin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 99: 100%|██████████| 150/150 [00:01<00:00, 93.39it/s, v_num=1] 

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 150/150 [00:01<00:00, 92.40it/s, v_num=1]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


c:\Users\abhin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 38/38 [00:00<00:00, 130.44it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8308333158493042     │
│         test_loss         │    1.2794222831726074     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.2794222831726074, 'test_accuracy': 0.8308333158493042}]